File notebook này có tác dụng tiền xử lý file bao gồm hai công đoạn:




In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
df=pd.read_csv('../Datasets/air_quality.csv')

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df_filtered=df[df['county']=='Taipei City']


In [ ]:
df_filtered['sitename'].unique()

In [ ]:
df_filtered=df_filtered[df_filtered['sitename']=='Shilin']

In [ ]:
df_after_removecol = df_filtered.drop(columns=['latitude', 'longitude','siteid','unit','county'])

In [ ]:
data=df_after_removecol.dropna(subset=['pm2.5','co_8hr','no2','o3','so2','pm10','aqi','co','windspeed','winddirec','pm2.5_avg','o3_8hr','status'])
data['pollutant'] = data['pollutant'].fillna('Normal')

data.replace('ND', np.nan, inplace=True)

data.fillna(method='ffill', inplace=True)

In [ ]:

columns_to_convert = ['so2', 'co', 'o3', 'o3_8hr', 'pm10', 'pm2.5', 
                      'no2', 'nox', 'no', 'windspeed', 'winddirec', 
                      'co_8hr', 'pm2.5_avg', 'pm10_avg', 'so2_avg']

# Sử dụng astype để chuyển tất cả các cột sang kiểu float
for col in columns_to_convert:
    data[col]=data[col].astype(float)

In [ ]:
print(data['date'].dtypes)


In [ ]:
data.loc[:, 'date'] = data['date'].astype(str)
data.loc[:, 'date'] = data['date'].str.replace("/", "-", regex=False)


In [ ]:
data['date'] = pd.to_datetime(data['date'], format="%Y-%m-%d %H:%M", errors='coerce')
data = data.dropna(subset=['date'])

In [ ]:
dtypes = data.dtypes
float_cols = set(dtypes[(dtypes==np.float32) | (dtypes==np.float64)].index)

In [ ]:
try:
    distributed_rows = ['missing_ratio', 'min', 'lower_quartile', 'median', 'upper_quartile', 'max']
    distributed_columns = list(float_cols)  
    
    distributed_value = []


    for numeric_data in float_cols:
        distributed_value.append([
            data[numeric_data].isnull().mean() * 100,  
            data[numeric_data].min(),  
            data[numeric_data].quantile(0.25),  
            data[numeric_data].median(),  
            data[numeric_data].quantile(0.75),  
            data[numeric_data].max()  
        ])
    
    num_col_info_df = pd.DataFrame(distributed_value, index=distributed_columns, columns=distributed_rows).T.round(1)
except:
    raise NotImplementedError()

In [ ]:
dict(num_col_info_df.iloc[0])

In [ ]:
num_col_info_df

In [ ]:
data.to_csv('../Datasets/air_quality_shilin.csv',index=False,sep=",")